# Capital Gain Calculator (FIFO Method)
Adapted from VBA to Python

### Structures
1. Transaction queue
    - HashMap (key is Asset, dict is tuple of two deques)
2. Transactions
    - 6 properties: Timestamp, Asset (eg. BTC), Type (buy or sell), Units, Total Amount ($), IRS ID (eg. Gemi 1)

### Outline
1. Validate transaction log CSV
    - Sort CSV by Timestamp property
    - Ensure valid Type property and corresponding Units property sign
    - Ensure other properties properties
2. Input data from transaction log CSV file into a buy transaction queue and sell transaction queue for each asset (eg. transaction log with BTC and ETH transactions -> BTC-Buy, BTC-Sell, ETH-Buy, Eth-Sell)
3. For each asset, run FIFO algorithm
    - While there are still transactions in the sell transaction queue, match front of sell queue with front of buy queue transaction (verifying buy transaction Timestamp property is before sell transaction)
    - When a match is found, add write buy-sell transaction to output CSV file
    - Update remaining balance (Units and Total Amount property) on buy/sell transaction and/or remove empty transaction(s) from respective queue
    - Run until the sell transaction queue is empty, the remaining buy transaction(s) are the carryover
4. Create summary report

# 1. Read and Validate Transaction Log CSV

In [467]:
import csv
import datetime

# Define a function converting a timestamp string (must be mm/dd/yyyy hh:mm:ss) into a datetime object
def timestamp(string):
    return datetime.datetime.strptime(string, "%m/%d/%Y %H:%M:%S")

# Open input CSV file
file_name = "large_example.csv"
file = open(file_name, 'r')
transaction_log = csv.DictReader(file)

# Sort transaction log by Timestamp property
transaction_log = sorted(transaction_log, key=lambda d: timestamp(d['Timestamp']))

# Change datatypes of Units and Total Amount to float
for tx in transaction_log:
    tx['Units'] = float(tx['Units'])
    tx['Total Amount'] = float(tx['Total Amount'])

# 2. Input Data into Transaction Queues

In [468]:
from collections import deque

# Create Transaction Queues
asset_map = {}

# Define buy and sell constants
BUY = 0
SELL = 1

for tx in transaction_log:
    
    # Create buy and sell deques for each asset
    asset = tx['Asset']
    if asset not in asset_map:
        buy_deque = deque()
        sell_deque = deque()
        asset_map[asset] = (buy_deque, sell_deque)
    
    # Add transaction to respective deque
    if tx['Type'] == "Buy":
        asset_map[asset][BUY].appendleft(tx)
    else:
        asset_map[asset][SELL].appendleft(tx)
        
# Close input CSV File
file.close()

# 3. Run FIFO Transaction Matching

In [469]:
# Open output CSV file
file_name = "fifo.csv"
file = open(file_name, 'w', encoding='UTF8', newline='')

# Define fieldnames
fieldnames = ['Asset', 'Date Purchased', 'Date Sold', 'Units', 'Sale Price', 'Basis',
              'Gain / Loss', 'IRS ID Buy', 'IRS ID Sell']
writer = csv.DictWriter(file, fieldnames=fieldnames)

# Write fieldnames into output CSV file
writer.writeheader()

# Create dictionary to store short and long term capital gain/loss statistics
year_summary = {}

# Volume statistics
capital_gain, buy_volume, sell_volume = 0, 0, 0

# Loop through each asset
for asset in asset_map:
   
    # Create dictionary to store match transaction information
    match = {}
    match['Asset'] = asset
    
    # Loop until there are no more transactions in sell deque
    while asset_map[asset][SELL]:
        
        # Get earliest non-matched sell transaction
        sell_tx = asset_map[asset][SELL].pop()
        
        # rounding down to 0????
        if abs(sell_tx['Units']) < .0000001:
            print("ID Sell:", sell_tx['IRS ID'], ",", "Remaining units:", sell_tx['Units'], "(consider as 0)")
            # continue statement skips remaining code and goes to next sell transaction
            continue
        
        # Catch error here: While sell deque is not empty, buy deque must not be empty
        assert(asset_map[asset][BUY])
        
        # Get earliest non-matched buy transaction
        buy_tx = asset_map[asset][BUY].pop()
        
        # Catch error here: Buy_tx['Timestamp'] must be before sell_tx['Timestamp']
        assert(timestamp(buy_tx['Timestamp']) <= timestamp(sell_tx['Timestamp']))
        
        # Populate match dictionary
        match['IRS ID Buy'] = buy_tx['IRS ID']
        match['IRS ID Sell'] = sell_tx['IRS ID']
        match['Date Purchased'] = buy_tx['Timestamp']
        match['Date Sold'] = sell_tx['Timestamp']
        
        # Sell transaction units are greater, so empty buy transaction
        if abs(sell_tx['Units']) > buy_tx['Units']:
            
            # Calculate pro rata sale price
            pro_rata_sale_price = buy_tx['Units'] / abs(sell_tx['Units']) * sell_tx['Total Amount']
            
            # Populate match dictionary
            match['Units'] = buy_tx['Units']
            match['Sale Price'] = pro_rata_sale_price
            match['Basis'] = buy_tx['Total Amount']
            
            # Update sell transaction information and put back into deque
            sell_tx['Units'] = sell_tx['Units'] + buy_tx['Units']
            sell_tx['Total Amount'] = sell_tx['Total Amount'] - pro_rata_sale_price
            asset_map[asset][SELL].append(sell_tx)
        
        # Buy transaction units are greater, so empty sell transaction
        elif abs(sell_tx['Units']) < buy_tx['Units']:
            
            # Calculate pro rata basis
            pro_rata_basis = abs(sell_tx['Units']) / buy_tx['Units'] * buy_tx['Total Amount']
            
            # Populate match dictionary
            match['Units'] = abs(sell_tx['Units'])
            match['Sale Price'] = sell_tx['Total Amount']
            match['Basis'] = pro_rata_basis
            
            # Update buy transaction information and put back into deque
            buy_tx['Units'] = buy_tx['Units'] + sell_tx['Units']
            buy_tx['Total Amount'] = buy_tx['Total Amount'] - pro_rata_basis
            asset_map[asset][BUY].append(buy_tx)
        
        # Transaction units are the same
        else:
            # Populate match dictionary
            match['Units'] = buy_tx['Units']
            match['Sale Price'] = sell_tx['Total Amount']
            match['Basis'] = buy_tx['Total Amount']
            
        # Calculate match gain or loss
        match['Gain / Loss'] = match['Sale Price'] - match['Basis']

        # Write transaction match into output CSV file
        writer.writerow(match)
        
        # Update year by year summary statistics
        year = timestamp(sell_tx['Timestamp']).year
        
        if year not in year_summary:
            year_summary[year] = {'Year': year, 'STCG': 0, 'STCL': 0, 'LTCG': 0, 'LTCL': 0, 'Net CG': 0}
        
        if timestamp(sell_tx['Timestamp']) - timestamp(buy_tx['Timestamp']) < datetime.timedelta(days=365):
            if match['Gain / Loss'] > 0:
                year_summary[year]['STCG'] += match['Gain / Loss']
            else:
                year_summary[year]['STCL'] += match['Gain / Loss']
        else:
            if match['Gain / Loss'] > 0:
                year_summary[year]['LTCG'] += match['Gain / Loss']
            else:
                year_summary[year]['LTCL'] += match['Gain / Loss']
                
        year_summary[year]['Net CG'] += match['Gain / Loss']
        
        # Update volume and total capital gain statistics
        capital_gain += match['Gain / Loss']
        sell_volume += match['Sale Price']
        buy_volume += match['Basis']

        
    # The asset carryover are the remaining transactions in the buy deque
    while asset_map[asset][BUY]:
        buy_tx = asset_map[asset][BUY].pop()
        
        # rounding down to 0????
        if abs(buy_tx['Units']) < .0000001:
            print("ID Buy:", buy_tx['IRS ID'], ",", "Remaining units:", buy_tx['Units'], "(consider as 0)")
            continue
        
        # Populate match as a carryover
        match['IRS ID Buy'] = buy_tx['IRS ID']
        match['IRS ID Sell'] = '-'
        match['Date Purchased'] = buy_tx['Timestamp']
        match['Date Sold'] = '-'
        match['Units'] = buy_tx['Units']
        match['Sale Price'] = '-'
        match['Basis'] = buy_tx['Total Amount']
        match['Gain / Loss'] = '-'
        
        # Write carryover into output CSV file
        writer.writerow(match)
        
        buy_volume += buy_tx['Total Amount']
        
volume = {'Asset': '-', 'Date Purchased': '-', 'Date Sold': '-', 'Units': '-', 'Sale Price': sell_volume,
          'Basis': buy_volume, 'Gain / Loss': capital_gain, 'IRS ID Buy': '-', 'IRS ID Sell': '-'}
writer.writerow(volume)

# Close output CSV File
file.close()

ID Sell: Gemi 21 , Remaining units: -6.661338147750939e-16 (consider as 0)
ID Sell: Gemi 1285 , Remaining units: -4.0000037504484e-08 (consider as 0)
ID Sell: Gemi 654 , Remaining units: -1.27675647831893e-15 (consider as 0)
ID Buy: Gemi 976 , Remaining units: 6.786238238021269e-14 (consider as 0)


# 4. Create Summary Report

In [470]:
# Open output CSV file for summary
file_name = "summary.csv"
file = open(file_name, 'w', encoding='UTF8', newline='')

# Define fieldnames
fieldnames = ['Year', 'STCG', 'STCL', 'LTCG', 'LTCL', 'Net CG']
writer = csv.DictWriter(file, fieldnames=fieldnames)

# Write fieldnames into output CSV file
writer.writeheader()

totals = {'Year': 'Totals', 'STCG': 0, 'STCL': 0, 'LTCG': 0, 'LTCL': 0, 'Net CG': 0}

for year in year_summary:
    writer.writerow(year_summary[year])
    totals['STCG'] += year_summary[year]['STCG']
    totals['STCL'] += year_summary[year]['STCL']
    totals['LTCG'] += year_summary[year]['LTCG']
    totals['LTCL'] += year_summary[year]['LTCL']
    totals['Net CG'] += year_summary[year]['Net CG']

writer.writerow(totals)

file.close()